# 01 — Data Audit (Trust & Definitions)

This notebook answers one question:
**Can we trust this dataset enough to analyze what drives music popularity?**

We will:
1. Verify schema + missingness + duplicates
2. Validate ranges + unit consistency (tempo, duration, loudness, etc.)
3. Define the target variable ("popularity") and known confounds
4. Produce a modeling-ready table and a data dictionary

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_raw = pd.read_csv("../data/raw/apple_music_dataset.csv")
track_df = df_raw.copy()

display(track_df.head())
display(track_df.tail())

,artistId,artistName,collectionCensoredName,collectionId,collectionName,collectionPrice,contentAdvisoryRating,country,currency,discCount,...,primaryGenreName,releaseDate,trackCensoredName,trackCount,trackExplicitness,trackId,trackName,trackNumber,trackPrice,trackTimeMillis
0,46087,Erick Sermon,Music,298321651,Music,9.99,Explicit,USA,USD,1,...,Hip-Hop/Rap,2001-08-27T12:00:00Z,Music (feat. Marvin Gaye),16,explicit,298321904,Music,4,1.29,223133
1,20044,Madonna,Music,80815197,Music,9.99,NaN,USA,USD,1,...,Pop,2000-08-21T07:00:00Z,Music,10,notExplicit,80815173,Music,1,1.29,225973
2,486597,Journey,Greatest Hits (2024 Remaster),169003304,Greatest Hits (2024 Remaster),10.99,NaN,USA,USD,1,...,Rock,1981-06-03T07:00:00Z,Don't Stop Believin' (2024 Remaster),16,notExplicit,169003415,Don't Stop Believin' (2024 Remaster),2,1.29,250835
3,156987,Jason Mraz,We Sing. We Dance. We Steal Things,277635758,We Sing. We Dance. We Steal Things,11.99,NaN,USA,USD,1,...,Pop,2008-02-12T08:00:00Z,I'm Yours,12,notExplicit,277635828,I'm Yours,2,1.29,242947
4,46087,Erick Sermon,Music,298429528,Music,9.99,Clean,USA,USD,1,...,Hip-Hop/Rap,2001-05-29T07:00:00Z,Music (feat. Marvin Gaye),16,cleaned,298429596,Music,4,1.29,223133


,artistId,artistName,collectionCensoredName,collectionId,collectionName,collectionPrice,contentAdvisoryRating,country,currency,discCount,...,primaryGenreName,releaseDate,trackCensoredName,trackCount,trackExplicitness,trackId,trackName,trackNumber,trackPrice,trackTimeMillis
9995,5192163,Dave Matthews & Tim Reynolds,Live At Radio City (Bonus Track Version),261552981,Live At Radio City (Bonus Track Version),12.99,NaN,USA,USD,2,...,Rock,2007-08-14T12:00:00Z,So Damn Lucky (Live),14,notExplicit,261553091,So Damn Lucky,6,1.29,411640
9996,214135551,Red,Until We Have Faces,416155893,Until We Have Faces,9.99,NaN,USA,USD,1,...,Rock,2011-02-01T12:00:00Z,Faceless,11,notExplicit,416155943,Faceless,2,1.29,203520
9997,121582,The Isley Brothers,The Essential Isley Brothers,198017385,The Essential Isley Brothers,14.99,NaN,USA,USD,2,...,R&B/Soul,1973-07-14T07:00:00Z,"That Lady, Pts. 1 & 2",18,notExplicit,198017838,"That Lady, Pts. 1 & 2",2,1.29,334387
9998,91157642,David Eisley & Bob Kulick,Spongebob Squarepants - The Yellow Album,283379708,Spongebob Squarepants - The Yellow Album,9.99,NaN,USA,USD,1,...,Children's Music,2005-11-15T12:00:00Z,Sweet Victory,23,notExplicit,283379767,Sweet Victory,2,1.29,126653
9999,540530,Pat Benatar,Greatest Hits,715881927,Greatest Hits,9.99,NaN,USA,USD,1,...,Rock,1979-08-27T07:00:00Z,Heartbreaker (2005 Remaster),20,notExplicit,715882411,Heartbreaker (2005 Remaster),1,1.29,208307


# Fetching Data
- get popularity data from spotify API

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

assert CLIENT_ID is not None
assert CLIENT_SECRET is not None

## GEt Track Info

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm # Progress bar

URI = 'http://127.0.0.1:9090/callback'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=URI,
    scope="user-read-private"
))

def get_spotify_details(row):
    artist = str(row['artistName']).replace("'", "")
    track = str(row['trackName']).replace("'", "")
    query = f"track:{track} artist:{artist}"
    
    try:
        results = sp.search(q=query, type='track', limit=1)
        items = results['tracks']['items']
        
        if not items:
            return None 
        
        t = items[0]
        
        return {
            "spotify_id": t['id'],
            "spotify_artist_id": t['artists'][0]['id'],
            "spotify_popularity": t['popularity'],
            "spotify_explicit": t['explicit'],
            "cover_art_url": t['album']['images'][0]['url'] if t['album']['images'] else None,
            "spotify_release_date": t['album']['release_date']
        }
        
    except Exception as e:
        return None

print("⏳ Searching Spotify...")
tqdm.pandas()
spotify_data = track_df.progress_apply(get_spotify_details, axis=1)


clean_data = [x if x is not None else {} for x in spotify_data]
df_spotify_features = pd.DataFrame(clean_data)

# Merge
final_df = pd.concat([track_df.reset_index(drop=True), df_spotify_features], axis=1)

print(f"✅ Match Success Rate: {final_df['spotify_id'].notna().mean():.2%}")
display(final_df.head())

⏳ Searching Spotify...


  0%|          | 11/10000 [00:02<45:09,  3.69it/s]


KeyboardInterrupt: 

## Get Artist Info

In [ ]:
print(f"🎤 Artists to fetch: {final_df['spotify_artist_id'].nunique()}")

unique_ids = final_df['spotify_artist_id'].dropna().unique().tolist()

batch_size = 50
artist_data = []

print("Fetching Artist Popularity & Other Info...")
for i in tqdm(range(0, len(unique_ids), batch_size)):
    batch = unique_ids[i:i + batch_size]
    
    try:
        response = sp.artists(batch)
        for artist in response['artists']:
            image_url = artist['images'][0]['url'] if artist['images'] else None
            
            artist_data.append({
                "spotify_artist_id": artist['id'],
                "artist_popularity": artist['popularity'],
                "artist_genres": artist['genres'],
                "artist_followers": artist['followers']['total'],
                "artist_image_url": image_url  # <--- NEW FIELD ADDED HERE
            })
    except Exception as e:
        print(f"Error on batch {i}: {e}")

df_artists_only = pd.DataFrame(artist_data)
final_df = final_df.merge(df_artists_only, on="spotify_artist_id", how="left")

print("✅ Added Artist Info!")
display(final_df[['trackName', 'artist_popularity']].head())

🎤 Artists to fetch: 1652
Fetching Artist Popularity & Other Info...


100%|██████████| 34/34 [00:07<00:00,  4.62it/s]

✅ Added Artist Info!


,trackName,artist_popularity
0,Music,52.0
1,Music,96.0
2,Don't Stop Believin' (2024 Remaster),76.0
3,I'm Yours,74.0
4,Music,52.0


In [ ]:
df = final_df.copy()
del final_df

In [ ]:
rename_map = {
    "artistId": "apple_artist_id",
    "trackId": "apple_track_id",
    "artistName": "artist_name",
    "trackName": "track_name",
    "primaryGenreName": "genre",
    "releaseDate": "release_date",
    "trackTimeMillis": "track_time_ms",
    "previewUrl": "preview_url",
    "contentAdvisoryRating": "advisory_rating",
    "collectionId": "apple_collection_id",
    "collectionName": "collection_name",
    "trackExplicitness": "track_explicitness",
    "country": "country",
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

In [ ]:
base_cols = [
    "apple_track_id", "apple_artist_id",
    "track_name", "artist_name",
    "genre", "country",
    "release_date", "release_year", "release_month", "release_ym",
    "track_time_ms", "track_time_min",
    "preview_url", "advisory_rating", "track_explicitness",
    "apple_collection_id", "collection_name",
]
base_cols = [c for c in base_cols if c in df.columns]
base = df[base_cols].copy()

In [ ]:
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["release_year"] = df["release_date"].dt.year
df["release_month"] = df["release_date"].dt.month
df["release_ym"] = df["release_date"].dt.to_period("M").astype(str)

df["track_time_ms"] = pd.to_numeric(df["track_time_ms"], errors="coerce")
df["track_time_min"] = df["track_time_ms"] / 60000

/var/folders/fv/hnh2s54s4h9_vv9q4x1rbdj40000gn/T/ipykernel_21947/1787974535.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["release_ym"] = df["release_date"].dt.to_period("M").astype(str)


In [ ]:
base = base.drop_duplicates(subset=["apple_track_id"]).reset_index(drop=True)
base.to_parquet("../data/processed/base_tracks.parquet", index=False)

In [ ]:
spotify_cols = [
    "apple_track_id",
    "spotify_id", "spotify_artist_id",
    "spotify_popularity", "spotify_explicit",
    "spotify_release_date", "cover_art_url",
    "spotify_popularity_fetched_at",

    "match_confidence", "spotify_name", "spotify_artist_name",
]
spotify_cols = [c for c in spotify_cols if c in df.columns]
spotify_enrich = df[spotify_cols].copy()

spotify_enrich.to_parquet("../data/processed/spotify_enrichment.parquet", index=False)

In [ ]:
artist_cols = ["spotify_artist_id", "artist_followers", "artist_popularity", "artist_genres", "artist_image_url"]
artist_cols = [c for c in artist_cols if c in df.columns]
artist_enrich = (df[artist_cols]
                .dropna(subset=["spotify_artist_id"])
                .drop_duplicates(subset=["spotify_artist_id"])
                .reset_index(drop=True))
artist_enrich.to_parquet("../data/processed/artist_enrichment.parquet", index=False)